In [0]:
! pip install kashgari
! pip install pandas
! pip install tensorflow==1.15.0

In [0]:
%tensorflow_version 1.x


In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [0]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding
import codecs
import csv
import pandas as pd
import ast
# kashgari.config.use_cudnn_cell = True


In [0]:
import os
os.getcwd()
! ls

In [0]:
import zipfile
zfile = zipfile.ZipFile("drive/My Drive/albert.zip")
zfile.extractall("albert")
!ls

In [0]:
#引入問題跟答案
df_sen = pd.read_csv("result_2.csv",encoding="utf-8")

#建立問題與答案list以符合訓練格式
all_ans = []
all_sen = []
aspectTerm_t = []
for i in range(df_sen.shape[0]):
    a = str(df_sen.loc[i, "dish"]).split("?")
    s = str(df_sen.loc[i, "text"])
    # t = str(df_sen.loc[i,"ans"]).split("?")
    one_ans = []
    #  <aspectTerm> : from="4" polarity="positive" term="food" to="8"
    aspectTerm_i = []
    for i in range(len(s)):
        one_ans = one_ans + ["O"]
    # print(s)
    # print(a)
    for j in a:
        try:
            jj = j.split(",")
            j1 = jj[0]
            j2 = jj[1].replace(" a", "positive").replace("a豆干肉絲", "positive").replace("a", "positive").replace("b",
                                                                                                               "neutral").replace(
                "c", "negative")
        except:
            continue
        if j1 in s:
            j1_from = s.index(j1)
            j1_to = s.index(j1) + len(j1)
            aspectTerm_p = [j1, str(j1_from), str(j1_to), j2]
            # print(aspectTerm_p)
            aspectTerm_i = aspectTerm_i + [aspectTerm_p]
            one_ans[j1_from] = "B-1"
            for x in range(len(j1) - 1):
                one_ans[s.index(j1) + x + 1] = "I-1"
    all_ans = all_ans + [one_ans]
    all_sen = all_sen + [s]
    # print(one_ans)

    aspectTerm_t = aspectTerm_t + [aspectTerm_i]

all_sen_1 = []
for all in range(len(all_sen)):
    y=[]
    for i in all_sen[all]:
        y.append(i)
    all_sen_1.append(y)

#all_sen_1 =全部問題,all_ans=全部答案
print(all_sen_1)
print(all_ans)

[['炒', '泡', '麵', '份', '量', '十', '足', '好', '吃', '，', '豆', '腐', '也', '很', '美', '味', '，', '熱', '炒', '類', '都', '蠻', '十', '足', '美', '味', '非', '常', '平', '價'], ['2', '0', '1', '9', '0', '3', '0', '7', '我', '點', '牛', '肉', '燴', '，', '讚', '。', '，', '，', '還', '不', '錯', '啊', '！', '我', '點', '的', '是', '肉', '絲', '燴', '飯', '，', '料', '多', '味', '美', '，', '雖', '然', '偏', '鹹', '了', '一', '點', '，', '但', '還', '可', '以', '接', '受', '，', '是', '有', '回', '雇', '率', '的', '店', '，', '重', '點', '香', '氣', '又', '十', '足', '，', '價', '格', '又', '公', '道', '。'], ['牛', '肉', '炒', '飯', '，', '飯', '粒', '粒', '分', '明', '，', '口', '味', '偏', '鹹', '，', '但', '好', '吃', '！'], ['上', '菜', '速', '度', '實', '在', '太', '慢', '，', '蒜', '泥', '白', '肉', '不', '錯', ' ', '，', '龍', '珠', '普', '普', '，', '鳳', '梨', '蝦', '球', '讚', '，', '客', '家', '小', '炒', '還', '可', '以', '，', '鐵', '板', '肉', '類', '還', '不', '錯'], ['吃', '過', '三', '次', ' ', '豆', '腐', '好', '吃'], ['烤', '斑', '鳩', '好', '吃'], ['價', '格', '公', '道', '，', '鹹', '蛋', '杏', '鮑', '菇', '，', '炒', '螺', '肉', '很', '下', '

In [0]:
train_x =all_sen_1[0:2600]
train_y =all_ans[0:2600]
valid_x =all_sen_1[2601:2700]
valid_y =all_ans[2601:2700]
test_x =all_sen_1[2701:2876]
test_y =all_ans[2701:2876]

In [0]:
#設定一層embedding，bert訓練模型會給進去的每一個詞一個index及向量(包含位置向量),每一批次訓練會丟最多128個字進去訓練
#所以輸出最多也128個字
bert_embed = BERTEmbedding('drive/My Drive/robert',
                           task=kashgari.LABELING,
                           sequence_length=128)


from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.python import keras

#patience=3是看每一個epoch
stop_callback = EarlyStopping(patience=5, restore_best_weights=True)
# save_callback = ModelCheckpoint("530test1.h5",save_best_only=True,save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-6)


model = BiLSTM_CRF_Model(bert_embed)
model.fit(train_x,
          train_y,
          x_validate=valid_x,
          y_validate=valid_y,
          callbacks=[stop_callback,reduce_lr],
          batch_size=128,
          epochs=50)

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [0]:
# 驗證模型印出 precision、recall、f1
model.evaluate(test_x, test_y)

# 保存模型到 `use_this_model` 目錄下
model.save('ner_model')

# 加載保存模型
loaded_model = kashgari.utils.load_model('ner_model')

# 使用模進行預測
loaded.predict(test_x[0])


           precision    recall  f1-score   support

        1     0.8660    0.8825    0.8742       315

micro avg     0.8660    0.8825    0.8742       315
macro avg     0.8660    0.8825    0.8742       315



In [0]:
import pandas as pd
import kashgari
import re
import glob

#使用這個目錄下的模型
loaded_model = kashgari.utils.load_model('drive/Mydrive/final_model')

#定義函式取得預測的結果
class get_menu:

    def cut_text(text, lenth):
        textArr = re.findall('.{' + str(lenth) + '}', text)
        textArr.append(text[(len(textArr) * lenth):])
        return textArr


    def extract_labels(text, ners):
        ner_reg_list = []
        if ners:
            new_ners = []
            for ner in ners:
                new_ners += ner;
            for word, tag in zip([char for char in text], new_ners):
                if tag != 'O':
                    ner_reg_list.append((word, tag))

# 输出模型的NER识别结果
        labels = {}
        if ner_reg_list:
            for i, item in enumerate(ner_reg_list):
                if item[1].startswith('B'):
                    label = ""
                    end = i + 1
                    while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I'):
                        end += 1

                    ner_type = item[1].split('-')[1]

                    if ner_type not in labels.keys():
                        labels[ner_type] = []

                    label += ''.join([item[0] for item in ner_reg_list[i:end]])
                    labels[ner_type].append(label)

        return labels

In [0]:
#建立一個能讓dataframe中的評論欄位,能預測菜單的函式
def get_name(sentence):

    text_1 = get_menu.cut_text(sentence, 100)
    ners = loaded_model.predict([[char for char in text] for text in text_1])
    labels = get_menu.extract_labels(sentence, ners)
    x = labels.get("1","")
    return ",".join(x)


In [0]:
import time

#取得所有檔案
get_filename = glob.glob("drive/My Drive/all_file/*")
trans = []
print(get_filename)

#轉換檔案加入tag欄位預測菜名
for i in get_filename:
    start = time.time()
    b = i
    a = i.replace("drive/My Drive/all_file/","")
    print(a," 讀取成功")
    df_sen = pd.read_csv(f"{b}",encoding="utf-8")
    print(df_sen.iloc[0:1,:])
    df_sen["tag"] = df_sen["text"].apply(get_name)
    print(df_sen.iloc[0:1,:])
    df_sen.to_csv("add_tag/{}+{}".format("T",a),index= False)
    print("T"+a," 寫入成功")
    end = time.time()
    running_time = (end-start)
    print('time cost : %.5f sec' %running_time)

['drive/My Drive/all_file/黑白炒_review.csv', 'drive/My Drive/all_file/海王子平價現炒_review.csv', 'drive/My Drive/all_file/阿文川味熱炒_review.csv', 'drive/My Drive/all_file/精武路菜脯雞_review.csv', 'drive/My Drive/all_file/阿忠澎湖野生海產_review.csv', 'drive/My Drive/all_file/佳鄉食堂_review.csv']
黑白炒_review.csv  讀取成功
   id name        date  star                                 text  dish
0   1  黑白炒  2014/05/26     5  "東西實在，價格公道，服務超棒，海產類的超新鮮，每一道菜都是真材實料"   NaN
   id name        date  star                                 text  dish tag
0   1  黑白炒  2014/05/26     5  "東西實在，價格公道，服務超棒，海產類的超新鮮，每一道菜都是真材實料"   NaN    
T黑白炒_review.csv  寫入成功
time cost : 8.97357 sec
海王子平價現炒_review.csv  讀取成功
   id     name        date  star text  dish
0   1  海王子平價現炒  2015/05/26     5  "讚"   NaN
   id     name        date  star text  dish tag
0   1  海王子平價現炒  2015/05/26     5  "讚"   NaN    
T海王子平價現炒_review.csv  寫入成功
time cost : 8.83985 sec
阿文川味熱炒_review.csv  讀取成功
   id    name        date  star            text  dish
0   1  阿文川味熱炒  2017/05/26     3

In [0]:
import glob
import time
import re

#使用這個目錄下的模型
loaded_model = kashgari.utils.load_model('drive/My Drive/rbt3_model')

#定義函式取得預測的結果
class get_dishes:

    global loaded_model

    def __init__(self,df):
        self.D = df

    def cut_text(self,text, lenth):
        textArr = re.findall('.{' + str(lenth) + '}', text)
        textArr.append(text[(len(textArr) * lenth):])
        return textArr


    def extract_labels(self,text, ners):
        ner_reg_list = []
        if ners:
            new_ners = []
            for ner in ners:
                new_ners += ner;
            for word, tag in zip([char for char in text], new_ners):
                if tag != 'O':
                    ner_reg_list.append((word, tag))

# 输出模型的NER识别结果
        labels = {}
        if ner_reg_list:
            for i, item in enumerate(ner_reg_list):
                if item[1].startswith('B'):
                    label = ""
                    end = i + 1
                    while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I'):
                        end += 1

                    ner_type = item[1].split('-')[1]
                    if ner_type not in labels.keys():
                        labels[ner_type] = []

                    label += ''.join([item[0] for item in ner_reg_list[i:end]])
                    labels[ner_type].append(label)

        return labels

#建立一個能讓dataframe中的評論欄位,能預測菜單的函式
    def get_name(self,sentence):

        text_1 = self.cut_text(sentence, 100)
        ners = loaded_model.predict([[char for char in text] for text in text_1])
        labels = self.extract_labels(sentence, ners)
        x = labels.get("1","")
        return ",".join(x)

#新增一個tag欄位，放入預測的菜單
    def Get(self):
        self.D = self.D.drop(["dish"],axis=1)
        self.D["dish"] = self.D["text"].apply(self.get_name)
        self.D = self.D[self.D["dish"].str.len()>0]
        return self.D

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [0]:
df_sen = pd.read_csv("阿文川味熱炒_review.csv",encoding="utf-8")
start = time.time()

data_frame = get_dishes(df_sen)
print(data_frame.Get())

data_frame.Get().to_csv("RoBERTa_阿文")
end = time.time()
running_time = (end-start)
print('time cost : %.5f sec' %running_time)

    id  ...                                dish
5    6  ...       客家小炒,肉絲,牛肉燴飯,青椒牛肉炒飯,鳳梨蝦球,五更腸旺
10  11  ...                               炒麵,炒飯
20  21  ...                                 蛤蜊湯
24  25  ...                                  炒飯
25  26  ...  炒飯,大酸辣湯,青椒牛肉,宮保雞丁,卡蘭蔬菜,煎蛋煎餅,蒸魚,炸豆腐
26  27  ...                             豆酥鱈魚,炒飯
40  41  ...                            咖哩炒飯,蚵仔湯
43  44  ...                         鮮蚵油條,小菜苦瓜鳳梨
47  48  ...                               吻魚蛋炒飯

[9 rows x 6 columns]
time cost : 23.33132 sec
